In [1]:
import pandas as pd

In [2]:
lemma_result = pd.read_csv("../Clean/lemma_result.csv") #Dataset

In [3]:
print(lemma_result.head())

                                      cleaned_review  Label
0  one reviewers mention watch 1 oz episode hook ...      1
1  wonderful little production film technique una...      1
2  think wonderful way spend time hot summer week...      1
3  basically family little boy jake think zombie ...      0
4  petter mattei love time money visually stun fi...      1


In [4]:
print(lemma_result.shape)

(1000, 2)


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split
import random
import re

In [6]:
max_fatures = 7000 #จำนวนคำที่ใช้ใน model
tokenizer = Tokenizer(num_words=max_fatures, split=' ') 
tokenizer.fit_on_texts(lemma_result['cleaned_review'].values)
X1 = tokenizer.texts_to_sequences(lemma_result['cleaned_review'].values)
X1 = pad_sequences(X1)

In [7]:
Y1 = pd.get_dummies(lemma_result['Label']).values
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, random_state = 42)
print(X1_train.shape,Y1_train.shape)
print(X1_test.shape,Y1_test.shape)

(750, 580) (750, 2)
(250, 580) (250, 2)


In [8]:
embed_dim = 150
lstm_out = 200

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X1.shape[1]))
#model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 580, 150)          1050000   
_________________________________________________________________
lstm (LSTM)                  (None, 200)               280800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 1,331,001
Trainable params: 1,331,001
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
Y1_train[:,0]

array([1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,

In [10]:
batch_size = 32
random.seed(10)
history = model.fit(X1_train, Y1_train[:,0],validation_split=0.2, epochs = 50, batch_size=batch_size, verbose = 2)

Train on 600 samples, validate on 150 samples
Epoch 1/50
600/600 - 28s - loss: 0.6927 - accuracy: 0.4833 - val_loss: 0.6891 - val_accuracy: 0.5133
Epoch 2/50
600/600 - 22s - loss: 0.6211 - accuracy: 0.7900 - val_loss: 0.6131 - val_accuracy: 0.6733
Epoch 3/50
600/600 - 22s - loss: 0.3556 - accuracy: 0.9333 - val_loss: 0.5153 - val_accuracy: 0.7400
Epoch 4/50
600/600 - 24s - loss: 0.1235 - accuracy: 0.9750 - val_loss: 0.5872 - val_accuracy: 0.7067
Epoch 5/50
600/600 - 24s - loss: 0.0518 - accuracy: 0.9933 - val_loss: 0.8464 - val_accuracy: 0.7267
Epoch 6/50
600/600 - 24s - loss: 0.0812 - accuracy: 0.9850 - val_loss: 0.5940 - val_accuracy: 0.7400
Epoch 7/50
600/600 - 24s - loss: 0.0394 - accuracy: 0.9933 - val_loss: 0.6095 - val_accuracy: 0.7133
Epoch 8/50
600/600 - 21s - loss: 0.0476 - accuracy: 0.9967 - val_loss: 0.6461 - val_accuracy: 0.7867
Epoch 9/50
600/600 - 22s - loss: 0.0071 - accuracy: 0.9983 - val_loss: 0.9141 - val_accuracy: 0.7333
Epoch 10/50
600/600 - 19s - loss: 0.0064 - ac

In [11]:
score,acc = model.evaluate(X1_test, Y1_test[:,0], verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

250/250 - 1s - loss: 1.1090 - accuracy: 0.7600
score: 1.11
acc: 0.76


In [12]:
lstm_output= model.predict(X1_test)

lstm_output

array([[1.31124616e-01],
       [9.71493244e-01],
       [9.88988459e-01],
       [9.99956608e-01],
       [9.02622414e-04],
       [2.86261402e-05],
       [6.35275990e-02],
       [9.98986781e-01],
       [9.96480405e-01],
       [2.60960072e-01],
       [4.51727479e-04],
       [6.12706244e-01],
       [9.99462068e-01],
       [2.06139013e-02],
       [9.94938612e-01],
       [9.93957043e-01],
       [1.61923053e-05],
       [6.41091645e-01],
       [4.00579542e-01],
       [2.06065553e-04],
       [1.08403729e-05],
       [9.97206151e-01],
       [9.99894619e-01],
       [2.00586044e-03],
       [9.99979496e-01],
       [9.59381580e-01],
       [9.92388427e-01],
       [2.29679481e-05],
       [2.09311515e-01],
       [7.93596730e-03],
       [9.99835849e-01],
       [9.99719083e-01],
       [2.54704431e-02],
       [1.69910002e-03],
       [9.99425173e-01],
       [1.12655351e-03],
       [1.30943224e-01],
       [4.08798410e-03],
       [9.97288465e-01],
       [1.68719143e-05],


In [13]:
print(X1_test)

[[   0    0    0 ...  278  287 2640]
 [   0    0    0 ...  525  436  116]
 [   0    0    0 ... 5459 4154   35]
 ...
 [   0    0    0 ... 5990 1056 2752]
 [   0    0    0 ...  163 1379  333]
 [   0    0    0 ...   21  438  505]]


In [14]:
import tensorflow as tf
confusion = tf.math.confusion_matrix(labels=Y1_test[:,0], predictions=lstm_output>0.5)

confusion

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[98, 23],
       [37, 92]])>

In [1]:
from sklearn.metrics import confusion_matrix
import pylab as plt
labels = ['positive','negative']
cm = confusion_matrix(Y1_test[:,0], lstm_output>0.5)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of LSTM classifier (1 dropout)')
fig.colorbar(cax)
ax.set_xticklabels(['']+labels)
ax.set_yticklabels(['']+labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

NameError: name 'Y1_test' is not defined

In [2]:
test = tokenizer.texts_to_sequences(lemma_result['cleaned_review'].values)
reverse_word_map = dict(map(reversed, tokenizer.word_index.items())) # map id to all word in dic
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

my_texts = list(map(sequence_to_text, X1_test))
my_texts[13]

print(reverse_word_map)

NameError: name 'tokenizer' is not defined